<a href="https://colab.research.google.com/github/nhatduong01/SQL_on_BigQuery/blob/main/Advance/Nested_and_Repeated_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will work on the `github_repos` dataset from `bigquery`

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
# set up the environment
import os
os.environ["GCLOUD_PROJECT"] = "BigQuerySQLExercises"

In [3]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

data_ref = client.dataset(dataset_id= 'github_repos', project= 'bigquery-public-data')

dataset = client.get_dataset(data_ref)

tables = list(client.list_tables(dataset))

for table in tables:
  print(table.table_id)

commits
contents
files
languages
licenses
sample_commits
sample_contents
sample_files
sample_repos


In [ ]:
# Construct a reference to `sample_commits` table
table_ref = dataset.table('sample_commits')

sample_commits = client.get_table(table_ref)

client.list_rows(sample_commits, max_results= 5).to_dataframe()

,commit,tree,parent,author,committer,subject,message,trailer,difference,difference_truncated,repo_name,encoding
0,3eca86e75ec7a7d4b9a9c8091b11676f7bd2a39f,8e1b4380409a85a922ee0d3f622b5dd4d16bcfad,[104a0c02e8b1936c049e18a6d4e4ab040fb61213],"{'name': 'Mark Rutland', 'email': '1db9dd262be...","{'name': 'Catalin Marinas', 'email': '15ce75b2...",arm64: Remove fixmap include fragility,arm64: Remove fixmap include fragility\n\nThe ...,"[{'key': 'Signed-off-by', 'value': 'Mark Rutla...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",None,torvalds/linux,None
1,7158627686f02319c50c8d9d78f75d4c8d126ff2,3b4d781bd966f07cad1b67b137f0ff8b89430e9a,[66aa8d6a145b6a66566b4fce219cc56c3d0e01c3],"{'name': 'Will Deacon', 'email': 'b913f13ef92a...","{'name': 'Catalin Marinas', 'email': '15ce75b2...",arm64: percpu: implement optimised pcpu access...,arm64: percpu: implement optimised pcpu access...,"[{'key': 'Signed-off-by', 'value': 'Will Deaco...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",None,torvalds/linux,None
2,9732cafd9dc0206479be919baf0067239f0a63ca,c8878035ac9cb6dce592957f12dc1723a583989d,[f3c003f72dfb2497056bcbb864885837a1968ed5],"{'name': 'Jiang Liu', 'email': 'c745fa7b96fe79...","{'name': 'Catalin Marinas', 'email': '15ce75b2...","arm64, jump label: optimize jump label impleme...","arm64, jump label: optimize jump label impleme...","[{'key': 'Reviewed-by', 'value': 'Will Deacon ...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",None,torvalds/linux,None
3,4702abd3f9728893ad5b0f4389e1902588510459,32926e7c55ef585d9b9c174a0e5f9ed13ed6bf7e,[ddf28352b80c86754a6424e3a61e8bdf9213b3c7],"{'name': 'Nicolas Pitre', 'email': '408789a210...","{'name': 'Arnd Bergmann', 'email': 'f2c659f019...",ARM: mach-nuc93x: delete,ARM: mach-nuc93x: delete\n\nThis architecture ...,"[{'key': 'Signed-off-by', 'value': 'Nicolas Pi...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",None,torvalds/linux,None
4,57bd4b91a6cfc5bad4c5d829ef85293ea63643ea,2ffc2066eb7638e185663e9d849663403229d4e5,[f74c95c20bad8e183e41283475f68a3e7b247af4],"{'name': 'Ben Dooks', 'email': '1177f64998f284...","{'name': 'Ben Dooks', 'email': '1177f64998f284...",[ARM] S3C24XX: Movev udc headers to arch/arm/p...,[ARM] S3C24XX: Movev udc headers to arch/arm/p...,"[{'key': 'Signed-off-by', 'value': 'Ben Dooks ...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",None,torvalds/linux,None


### 1) Who has most commits in 2016 ?
We write a query returned a table having 2 columns:
- `committer_name` - contains the name of each individual with a commit (from 2016) in the table
- `num_commits` - shows the number of commits the individual has in the table (from 2016)

In [ ]:
query1 = """
         SELECT committer.name AS committer_name,
         COUNT (1) AS num_commits
         FROM `bigquery-public-data.github_repos.sample_commits`
         WHERE committer.date >= '2016-01-01'
         GROUP BY committer.name
         ORDER BY num_commits DESC"""
query_job1 = client.query(query1, project = 'bigquerysqlexercises')
most_commits = query_job1.to_dataframe()
most_commits

,committer_name,num_commits
0,Greg Kroah-Hartman,3545
1,David S. Miller,3120
2,TensorFlower Gardener,2449
3,Linus Torvalds,2424
4,Benjamin Pasero,1127
...,...,...
748,Ivan,1
749,Danijar Hafner,1
750,Josh Perez,1
751,nakazye,1


We will work with the `languages` table now

In [4]:
table_ref = dataset.table('languages')

languages_table = client.get_table(table_ref)

client.list_rows(languages_table, max_results= 5).to_dataframe()

,repo_name,language
0,dhess/echoev,"[{'name': 'C', 'bytes': 82517}]"
1,bbidulock/vncproto,"[{'name': 'C', 'bytes': 7911}]"
2,Khalian/Karmarkar,"[{'name': 'C', 'bytes': 21738}]"
3,imheresamir/fillpipe,"[{'name': 'C', 'bytes': 5534}]"
4,zhaili/netcat,"[{'name': 'C', 'bytes': 105069}]"


In [5]:
# schema
languages_table.schema

[SchemaField('repo_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('language', 'RECORD', 'REPEATED', None, (SchemaField('name', 'STRING', 'NULLABLE', None, ()), SchemaField('bytes', 'INTEGER', 'NULLABLE', None, ())))]

### 2) What is the most popular programming language ?
We will write the query that returned 2 columns :
- `language_name` - the name of the programming language
- `num_repos` - the number of repositories in the `languages` table that use the programming language

In [8]:
query2 = """
         SELECT l.name AS language_name, COUNt(1)  AS num_repos
         FROM `bigquery-public-data.github_repos.languages`,
         UNNEST(language) AS l
         GROUP BY l.name
         ORDER BY num_repos DESC
         """
query_job2 = client.query(query2, project='bigquerysqlexercises')

popular_language = query_job2.to_dataframe()
popular_language.head(20)

,language_name,num_repos
0,JavaScript,1102846
1,CSS,816637
2,HTML,781560
3,Shell,639588
4,Python,550041
5,Ruby,375653
6,Java,369870
7,PHP,341134
8,C,293192
9,C++,279568


### 3) What languge is used in the repo with the most language ?
We will pay attention to the `'polyrabbit/polyglot'` repo.
We will write a query that returns:
- `name` - the name of the programming language
- `bytes` - the total number of bytes of that programming language


In [29]:
query3 = """
         SELECT l.name AS name,
         SUM (l.bytes) AS bytes
         FROM `bigquery-public-data.github_repos.languages`,
         UNNEST (language) AS l
         WHERE repo_name = 'polyrabbit/polyglot'
         GROUP BY l.name
         ORDER BY bytes DESC
         """
query_job3 = client.query(query3, project = 'bigquerysqlexercises')

used_language = query_job3.to_dataframe()
used_language

,name,bytes
0,Lasso,834726
1,C,819142
2,Mercury,709952
3,Objective-C,495392
4,Game Maker Language,298131
...,...,...
211,XC,82
212,Arduino,81
213,Nimrod,43
214,AutoHotkey,23


### Distinguish the difference between `HAVING` and `WHERE`

HAVING | WHERE
-------|------
Filter rows from groups $\rightarrow$ We must use it with | Filter rows from the
 the `Group by` statement                                 | table $\rightarrow$ we can use it when we like 


